In [1]:
import matplotlib.pyplot as plt
from scipy.stats import kde
import numpy as np
import astropy
from astropy.table import Table, join
from astropy.io import ascii
import scipy.integrate as integrate
from astropy.cosmology import Planck15
from astropy import units as u
import glob
import os

In [2]:
class Halos():
    
    def __init__(self, data, ScaleFactor):
        self.data = data
        self.redshift = (1/ScaleFactor) - 1 
        
        
    
    def categorise_galaxies(self):
        '''
        Finds masks to separate star forming and quiescent galaxies
        '''
        #finding redshift
        z = self.redshift 
        #the line from Aird(2021)
        log_sfrcut = -8.9 + 0.76*np.log10(self.data['sm']) + 2.95*np.log10(1+z)
        #finding the difference in y(sfr) value from line
        dsfr = np.log10(self.data['sfr']) - log_sfrcut

        #sf is when this difference is positive and qu is when this difference is negative
        
        sf = (dsfr >=0.0)
        qu = (dsfr<0.0)
        
        return sf, qu
    
    
    
    #adding bhm for z=0 first, this will be different for further redshifts so maybe later do if clauses (only if this will work with the id matching)
    def add_bhm_0(self, bhm_qu, bhm_sf, qu, sf):
        self.data.add_column(0.0, name='bhm')

        self.data['bhm'][qu] = bhm_qu
        self.data['bhm'][sf] = bhm_sf
        
        return self.data
    
    
    
    def find_Lbol(self, exp_l_array):
        
        m9_9 = (self.data['sm'] <= 10**(9.5))
        m9_10 = (self.data['sm'] > 10**(9.5)) & (self.data['sm'] <= (1e10))
        m10_10 = (self.data['sm'] > (1e10)) & (self.data['sm'] <= 10**(10.5))
        m10_11 = (self.data['sm'] > 10**10.5) & (self.data['sm'] <= 1e11)
        m11_11 = (self.data['sm'] > 1e11)

        self.data.add_column(0.0, name='Lbol')

        #finds Lbol for all bins and adds them to the halos table           
        bol9_9 = exp_l_array[0]*0.002*np.multiply((1.3E38),(self.data['sm'][m9_9]), dtype=np.float64)
        bol9_10 = exp_l_array[1]*0.002*np.multiply((1.3E38),(self.data['sm'][m9_10]), dtype=np.float64)
        bol10_10 = exp_l_array[2]*0.002*np.multiply((1.3E38),(self.data['sm'][m10_10]), dtype=np.float64)
        bol10_11 = exp_l_array[3]*0.002*np.multiply((1.3E38),(self.data['sm'][m10_11]), dtype=np.float64)
        bol11_11 = exp_l_array[4]*0.002*np.multiply((1.3E38),(self.data['sm'][m11_11]), dtype=np.float64)
        self.data['Lbol'][m9_9] = bol9_9
        self.data['Lbol'][m9_10] = bol9_10
        self.data['Lbol'][m10_10] = bol10_10
        self.data['Lbol'][m10_11] = bol10_11
        self.data['Lbol'][m11_11] = bol11_11
        
        return self.data

    
    
    def find_bhmgr(self):
        #finding black hole mass growth rate and adding it to the table
        constant = 3.154/(1.98847e30*(3e8**2)*0.1)

        bhmgr = self.data['Lbol']*constant
        self.data.add_column(0.0, name='bhmgr')
        self.data['bhmgr'] = bhmgr
        
        return self.data

    
    def find_lookback_time(self):
        z = self.redshift
        time_Gyr = astropy.cosmology.Planck15.lookback_time(z)
        time = time_Gyr *1e9/u.Gyr
        
        return time
    
    
    def find_bhm(self, halos_prior, dt):
        #finding matching ids

        #note minus sign so sorts from high to low mass
        sort = np.argsort(-self.data['mp'])         

        #sorted version of halos earlier
        self.data = self.data[sort]          

        #returns first match from halos_earlier_sorted to halos_later i.e. most massive progenitor
        matches, ind_earlier, ind_later = np.intersect1d(self.data['descid'], halos_prior.data['id'], return_indices=True)

        #then can add BH mass to halos_earlier_sorted, for the most massive progenitor
        self.data.add_column(0.0, name='bhm')
        self.data['bhm'][ind_earlier] = halos_prior.data['bhm'][ind_later] - halos_prior.data['bhmgr'][ind_later]*dt
        
        return self.data
    
    def no_zeros_mask(self, name):
        no_0 = (self.data[str(name)] != 0)
        
        return no_0
        

        

In [3]:
class Probabilities():
    def __init__(self, probs, lb_redshift):
        self.probs = probs
        self.lb_redshift = lb_redshift
        
    def expectation_lambda(self):
        #creating mask to extract only the redshift 0.1-0.5 data, and mass 9.0-9.5 data
        z = (self.probs['z_lo'] == self.lb_redshift)

        #creating mask to limit log(lambda) to less than 0 <- new fix
        log1 = (self.probs['logl'] < 0)

        #creating masks to extract the different mass ranges
        mp9_9 = (self.probs['mass_lo'] == 9.0)
        mp9_10 = (self.probs['mass_lo'] == 9.5)
        mp10_10 = (self.probs['mass_lo'] == 10.0)
        mp10_11 = (self.probs['mass_lo'] == 10.5)
        mp11_11 = (self.probs['mass_lo'] == 11.0)

        #creating different bins for the different mass ranges
        bin0 = z & mp9_9 & log1
        bin1 = z & mp9_10 & log1
        bin2 = z & mp10_10 & log1
        bin3 = z & mp10_11 & log1
        bin4 = z & mp11_11 & log1
        
        bin_list = [bin0, bin1, bin2, bin3, bin4]
        dloglambda = probs['logl'][1] - probs['logl'][0]
    
        #trying to find the expectation value of lamda
        exp_l_list = []
        for x in range(0, 5):
            p_l = self.probs['p(median)'][bin_list[x]]
            l = 10**(self.probs['logl'][bin_list[x]])
            exp = np.sum(l*p_l)*dloglambda
            setattr(self, 'exp_l'+str(x), exp)
            exp_l_list.append(exp)
            
        exp_l_array = np.asarray(exp_l_list)
        
        return exp_l_array
    

    

In [4]:
dtype = np.dtype(dtype=[('id', 'i8'),('descid','i8'),('upid','i8'),
                        ('flags', 'i4'), ('uparent_dist', 'f4'),
                        ('pos', 'f4', (6)), ('vmp', 'f4'), ('lvmp', 'f4'),
                        ('mp', 'f4'), ('m', 'f4'), ('v', 'f4'), ('r', 'f4'),
                        ('rank1', 'f4'), ('rank2', 'f4'), ('ra', 'f4'),
                        ('rarank', 'f4'), ('A_UV', 'f4'), ('sm', 'f4'), 
                        ('icl', 'f4'), ('sfr', 'f4'), ('obs_sm', 'f4'), 
                        ('obs_sfr', 'f4'), ('obs_uv', 'f4'), ('empty', 'f4')],
                 align=True)

In [12]:
files = ['sfr_catalog_0.668185.bin',
'sfr_catalog_0.673248.bin',
'sfr_catalog_0.678310.bin',
'sfr_catalog_0.683373.bin',
'sfr_catalog_0.690967.bin',
'sfr_catalog_0.698560.bin',
'sfr_catalog_0.706154.bin',
'sfr_catalog_0.713748.bin',
'sfr_catalog_0.721342.bin',
'sfr_catalog_0.728935.bin', 
'sfr_catalog_0.736529.bin',
'sfr_catalog_0.744123.bin',
'sfr_catalog_0.751717.bin',
'sfr_catalog_0.759310.bin',
'sfr_catalog_0.766904.bin',
'sfr_catalog_0.774498.bin',
'sfr_catalog_0.782092.bin',
'sfr_catalog_0.789685.bin',
'sfr_catalog_0.797279.bin',
'sfr_catalog_0.804873.bin',
'sfr_catalog_0.812467.bin',
'sfr_catalog_0.820060.bin',
'sfr_catalog_0.827654.bin',
'sfr_catalog_0.835248.bin',
'sfr_catalog_0.842842.bin',
'sfr_catalog_0.850435.bin',
'sfr_catalog_0.858029.bin',
'sfr_catalog_0.865623.bin',
'sfr_catalog_0.873217.bin',
'sfr_catalog_0.880810.bin',
'sfr_catalog_0.888404.bin',
'sfr_catalog_0.895998.bin',
'sfr_catalog_0.903592.bin',
'sfr_catalog_0.911185.bin',
'sfr_catalog_0.918779.bin',
'sfr_catalog_0.926373.bin',
'sfr_catalog_0.933967.bin',
'sfr_catalog_0.941560.bin',
'sfr_catalog_0.949154.bin',
'sfr_catalog_0.956748.bin',
'sfr_catalog_0.964342.bin',
'sfr_catalog_0.971935.bin',
'sfr_catalog_0.979529.bin',
'sfr_catalog_0.987123.bin',
'sfr_catalog_0.994717.bin']
files.reverse()
print(files)

['sfr_catalog_0.994717.bin', 'sfr_catalog_0.987123.bin', 'sfr_catalog_0.979529.bin', 'sfr_catalog_0.971935.bin', 'sfr_catalog_0.964342.bin', 'sfr_catalog_0.956748.bin', 'sfr_catalog_0.949154.bin', 'sfr_catalog_0.941560.bin', 'sfr_catalog_0.933967.bin', 'sfr_catalog_0.926373.bin', 'sfr_catalog_0.918779.bin', 'sfr_catalog_0.911185.bin', 'sfr_catalog_0.903592.bin', 'sfr_catalog_0.895998.bin', 'sfr_catalog_0.888404.bin', 'sfr_catalog_0.880810.bin', 'sfr_catalog_0.873217.bin', 'sfr_catalog_0.865623.bin', 'sfr_catalog_0.858029.bin', 'sfr_catalog_0.850435.bin', 'sfr_catalog_0.842842.bin', 'sfr_catalog_0.835248.bin', 'sfr_catalog_0.827654.bin', 'sfr_catalog_0.820060.bin', 'sfr_catalog_0.812467.bin', 'sfr_catalog_0.804873.bin', 'sfr_catalog_0.797279.bin', 'sfr_catalog_0.789685.bin', 'sfr_catalog_0.782092.bin', 'sfr_catalog_0.774498.bin', 'sfr_catalog_0.766904.bin', 'sfr_catalog_0.759310.bin', 'sfr_catalog_0.751717.bin', 'sfr_catalog_0.744123.bin', 'sfr_catalog_0.736529.bin', 'sfr_catalog_0.7289

In [5]:
files = ['sfr_catalog_0.501122.bin',
'sfr_catalog_0.506185.bin',
'sfr_catalog_0.511247.bin',
'sfr_catalog_0.516310.bin',
'sfr_catalog_0.521372.bin',
'sfr_catalog_0.526435.bin',
'sfr_catalog_0.531497.bin',
'sfr_catalog_0.536560.bin',
'sfr_catalog_0.541622.bin',
'sfr_catalog_0.546685.bin'
]
files.reverse()
print(files)


['sfr_catalog_0.546685.bin', 'sfr_catalog_0.541622.bin', 'sfr_catalog_0.536560.bin', 'sfr_catalog_0.531497.bin', 'sfr_catalog_0.526435.bin', 'sfr_catalog_0.521372.bin', 'sfr_catalog_0.516310.bin', 'sfr_catalog_0.511247.bin', 'sfr_catalog_0.506185.bin', 'sfr_catalog_0.501122.bin']


In [ ]:
'sfr_catalog_0.551747.bin',
'sfr_catalog_0.556810.bin',
'sfr_catalog_0.561872.bin',
'sfr_catalog_0.566935.bin',
'sfr_catalog_0.571997.bin',
'sfr_catalog_0.577060.bin',
'sfr_catalog_0.582123.bin',
'sfr_catalog_0.587185.bin',
'sfr_catalog_0.592248.bin',
'sfr_catalog_0.597310.bin',
'sfr_catalog_0.602373.bin',
'sfr_catalog_0.607435.bin',
'sfr_catalog_0.612498.bin',
'sfr_catalog_0.617560.bin',
'sfr_catalog_0.622623.bin',
'sfr_catalog_0.627685.bin',
'sfr_catalog_0.632748.bin',
'sfr_catalog_0.637810.bin',
'sfr_catalog_0.642873.bin',
'sfr_catalog_0.647935.bin',
'sfr_catalog_0.652998.bin',
'sfr_catalog_0.658060.bin',
'sfr_catalog_0.663123.bin'

In [11]:
files = ['sfr_catalog_0.404935.bin',
'sfr_catalog_0.409997.bin',
'sfr_catalog_0.415060.bin',
'sfr_catalog_0.420122.bin',
'sfr_catalog_0.425185.bin',
'sfr_catalog_0.430247.bin',
'sfr_catalog_0.435310.bin',
'sfr_catalog_0.440372.bin',
'sfr_catalog_0.445435.bin',
'sfr_catalog_0.450497.bin',
'sfr_catalog_0.455560.bin',
'sfr_catalog_0.460622.bin',
'sfr_catalog_0.465685.bin',
'sfr_catalog_0.470747.bin',
'sfr_catalog_0.475810.bin',
'sfr_catalog_0.480872.bin',
'sfr_catalog_0.485935.bin',
'sfr_catalog_0.490997.bin',
'sfr_catalog_0.496060.bin']
files.reverse()
print(files)


['sfr_catalog_0.496060.bin', 'sfr_catalog_0.490997.bin', 'sfr_catalog_0.485935.bin', 'sfr_catalog_0.480872.bin', 'sfr_catalog_0.475810.bin', 'sfr_catalog_0.470747.bin', 'sfr_catalog_0.465685.bin', 'sfr_catalog_0.460622.bin', 'sfr_catalog_0.455560.bin', 'sfr_catalog_0.450497.bin', 'sfr_catalog_0.445435.bin', 'sfr_catalog_0.440372.bin', 'sfr_catalog_0.435310.bin', 'sfr_catalog_0.430247.bin', 'sfr_catalog_0.425185.bin', 'sfr_catalog_0.420122.bin', 'sfr_catalog_0.415060.bin', 'sfr_catalog_0.409997.bin', 'sfr_catalog_0.404935.bin']


In [5]:
files = ['sfr_catalog_0.334060.bin',
'sfr_catalog_0.339122.bin',
'sfr_catalog_0.344185.bin',
'sfr_catalog_0.349247.bin',
'sfr_catalog_0.354310.bin',
'sfr_catalog_0.359372.bin',
'sfr_catalog_0.364435.bin',
'sfr_catalog_0.369497.bin',
'sfr_catalog_0.374560.bin',
'sfr_catalog_0.379622.bin',
'sfr_catalog_0.384685.bin',
'sfr_catalog_0.389747.bin',
'sfr_catalog_0.394810.bin',
'sfr_catalog_0.399872.bin']
files.reverse()
print(files)

['sfr_catalog_0.399872.bin', 'sfr_catalog_0.394810.bin', 'sfr_catalog_0.389747.bin', 'sfr_catalog_0.384685.bin', 'sfr_catalog_0.379622.bin', 'sfr_catalog_0.374560.bin', 'sfr_catalog_0.369497.bin', 'sfr_catalog_0.364435.bin', 'sfr_catalog_0.359372.bin', 'sfr_catalog_0.354310.bin', 'sfr_catalog_0.349247.bin', 'sfr_catalog_0.344185.bin', 'sfr_catalog_0.339122.bin', 'sfr_catalog_0.334060.bin']


In [11]:
z0 = Table.read("redshift0.ecsv", format='ascii.ecsv')
z0 = Halos(z0, 1)
setattr(z0, 'time', z0.find_lookback_time())
z_prior = z0

In [17]:
z668185 = Table.read("new0.668185.ecsv", format='ascii.ecsv')
z668185 = Halos(z668185, 0.668185)
setattr(z668185, 'time', z668185.find_lookback_time())
z_prior = z668185

In [6]:
z551747 = Table.read("new0.551747.ecsv", format='ascii.ecsv')
z551747 = Halos(z551747, 0.551747)
setattr(z551747, 'time', z551747.find_lookback_time())
z_prior = z551747

In [24]:
z_prior = z668185

In [12]:
z501122 = Table.read("new0.501122.ecsv", format='ascii.ecsv')
z501122 = Halos(z501122, 0.501122)
setattr(z501122, 'time', z501122.find_lookback_time())
z_prior = z501122

In [6]:
z404935 = Table.read("new0.404935.ecsv", format='ascii.ecsv')
z404935 = Halos(z404935, 0.404935)
setattr(z404935, 'time', z404935.find_lookback_time())
z_prior = z404935

In [7]:
#opening the probability data
file = open('pledd_qu.txt')

#turning the data into an astropy table
probs = astropy.table.Table(np.loadtxt(file), names=['z_lo', 'z_hi', 'mass_lo', 'mass_hi', 'logl', 'p(median)', 'p_lolim(90%CL)', 'p_uplim(90%CL)', 'flag'])
file.close()

In [13]:
#finding expectation lambdas for redshift bin specified - note once the lb is 1.5 the mass needs to be contrained to >10^9.5
probs0 = Probabilities(probs, 1.0)
exp_ls = probs0.expectation_lambda()

In [14]:
print(exp_ls)

[0.00038033 0.00055915 0.00202495 0.00158648 0.00064962]


In [15]:
#finding the next snapshot back
#z_prior = z0
folder_path = '/disk01/jaird/data/universemachine/dr1/BP/'
for filename in files: 
        halos = np.fromfile(os.path.join(folder_path, filename), dtype=dtype)

        IGNORE_FLAG = 2**4
        for x in halos:
            if (x['flags'] & IGNORE_FLAG):
                continue
            
        #creates mask to find values where stellar mass is greater than 10^9
        mask = (halos['sm']>(10**(9))) #should I be limiting these masses? should my tables have all masses?

        #creates new data array for halos with stellar mass >10^9
        halos = halos[mask]

        #turn data into table
        table = astropy.table.Table(halos)

        #finding the scalefactor from the filename
        scalefactor = float(filename.split('sfr_catalog_')[1].split('.bin')[0])
        
        #attribute scalefactor and data to halos object
        z_current = Halos(table, scalefactor)
        
        #finding lookback time
        setattr(z_current, 'time', z_current.find_lookback_time())
        #print(z_current.time)

        #finding change in time
        dt = z_current.time - z_prior.time
        #print(dt)

        #finding bhm
        z_current.find_bhm(z_prior, dt)

        #finding bhmgr for z994717
        z_current.find_Lbol(exp_ls)
        z_current.find_bhmgr()
        
        print('z_' + str(scalefactor) + 'bhm: ' + str(z_current.data['bhm'][0]))
        
        #write to file
        z_current.data.write('new'+str(scalefactor)+'.ecsv', format='ascii.ecsv', overwrite=True)
        
        #making this data the prior redshift to prepare for next snapshot
        z_prior = z_current
        
        


z_0.49606bhm: 15300144814.077307
z_0.490997bhm: 15298717254.286385
z_0.485935bhm: 15297334565.215538
z_0.480872bhm: 15295952949.961897
z_0.47581bhm: 15294572182.596209
z_0.470747bhm: 15293195629.957136
z_0.465685bhm: 15291845651.22837
z_0.460622bhm: 15290863691.5956
z_0.45556bhm: 15290085117.988377
z_0.450497bhm: 15289330273.511913
z_0.445435bhm: 15288581107.768332
z_0.440372bhm: 15287854054.916155
z_0.43531bhm: 15287257010.299175
z_0.430247bhm: 15286675616.227598
z_0.425185bhm: 15286123672.227356
z_0.420122bhm: 15285587204.680735
z_0.41506bhm: 15285052750.297457
z_0.409997bhm: 15284520016.512854
z_0.404935bhm: 15283988433.867863
